# Merton Jump Diffusion: Analytical solution

In this notebook, we show how to price European options with the analytical solution of the Merton jump diffusion model.

The value of an European option under the Merton jump diffusion model is given by

\begin{equation}
    V_{MJD} = \sum_{k = 1}^{\infty} \frac{\exp \left(- \lambda T e^{\mu_j + \frac{1}{2}\sigma_j^2} \right) \left( \lambda T e^{\mu_j + \frac{1}{2}\sigma_j^2} \right)^k  }{k!} V_{BS}(S, K, T, \sigma_k, r_k, q)
\end{equation}

where $S$ is the stock price, $K$ is the strike price, $T$ is the time to maturity, $q$ is the dividend rate, $V_{BS}(\Theta)$ is the value of the option under the Black-Scoles model with parameters $\Theta$, $\lambda$ is the rate of arrival of the jumps, $\mu_j$ is the mean of the jumps and $\sigma_j$ is the standard deviation of the jumps. The volatility to plug in the Black-Scholes model at each $k$, $\sigma_k$, is given by

\begin{equation}
    \sigma_k = \sqrt{\sigma^2 + k \frac{\sigma_j^2}{T}}
\end{equation}

where $\sigma$ is the stock volatility. The risk-free rate to plug in the Black-Scholes model at each $k$, $r_k$, is given by

\begin{equation}
    r_k = r - \lambda(e^{\mu_j + \frac{1}{2}\sigma_j^2} - 1) + \frac{k \left( \mu_j + \frac{1}{2}\sigma_j^2 \right)}{T} 
\end{equation}

where $r$ is the risk-free rate.

Of course, when implementing the solution, we truncate the infinite summation. As $k$ increases, the components to be added become smaller and smaller (with the factorial in the denominator). Therefore, we can use a maximum value of $k$ high enough for the additional components to be negligible. Alternatively, we can select a threshold below which additional components of the sum are not added. 

Let's implement the function.

In [1]:
import numpy as np
from scipy.stats import norm

# Function for analytical Black-Scholes price
def bs_price(S, K, T, r, q, sigma, call=True):
    d1 = (np.log(S/K) + (r - q + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if call:
        return S * np.exp(-q*T) * norm.cdf(d1) - K * np.exp(-r*T)* norm.cdf(d2)
    else:
        return K * np.exp(-r*T) * norm.cdf(-d2) - S * np.exp(-q*T) * norm.cdf(-d1)                 

# Function for analytical Merton jump diffusion price
def merton_jump_analytical(S, K, T, r, q, sigma, mu_j , sigma_j, lam, call=True, max_iter=100, stop_cond=1e-15):
    V = 0
    for k in range(max_iter):
        r_k = r - lam*(np.exp(mu_j + 0.5*sigma_j**2)-1) + (k*(mu_j + 0.5*sigma_j**2)) / T
        sigma_k = np.sqrt( sigma**2 + (k* sigma_j**2) / T)
        sum_k = (np.exp(-(np.exp(mu_j + 0.5*sigma_j**2))*lam*T) \
                * ((np.exp(mu_j + 0.5*sigma_j**2))*lam*T)**k / (np.math.factorial(k))) \
                * bs_price(S, K, T, r_k, q, sigma_k, call)
        V += sum_k
        if sum_k < stop_cond: # if the last added component is below the threshold, return the current sum
            return V
    return V # return the value of the option when the maximum value of k is reached

Now, let's define some parameters and compute the call and put values.

In [2]:
# Market parameters
T = 1      # maturity
S0 = 1     # spot price
K = 1.1    # strike price
r = 0.05   # risk-free interest rate
q = 0.02   # dividend rate

# Model parameter
sigma = 0.4 # volatility

# Model parameters for the jump part
mu_j = -0.1
sigma_j = 0.15
lmbda = 0.5

# Call value
Vc = merton_jump_analytical(S0, K, T, r, q, sigma, mu_j, sigma_j, lmbda, call=True, max_iter=100, stop_cond=1e-15)

# Put value
Vp = merton_jump_analytical(S0, K, T, r, q, sigma, mu_j, sigma_j, lmbda, call=False, max_iter=100, stop_cond=1e-15)

print('Call price: ' + str(round(Vc, 4)))
print('Put price:  ' + str(round(Vp, 4)))

Call price: 0.1362
Put price:  0.2023
